<a href="https://colab.research.google.com/github/FozilovDiyorbek/Qilingan-loyihalar/blob/main/Amaliyot_Toshkent_shahridagi_uylarni_narhini_bashorat_qilish_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Necessary Libraries

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000
...,...,...,...,...,...,...,...
7560,"город Ташкент, Яшнободский район, Городок Авиа...",Яшнободский,1,38,5,5,24500
7561,"город Ташкент, Яшнободский район, 1-й проезд А...",Яшнободский,2,49,1,4,32000
7562,"город Ташкент, Шайхантахурский район, Зульфиях...",Шайхантахурский,2,64,3,9,40000
7563,"город Ташкент, Мирзо-Улугбекский район, Буюк И...",Мирзо-Улугбекский,1,18,1,4,11000


Handling text columns

In [4]:
df['price'] = pd.to_numeric(df['price'], errors='coerce').astype('Int64')### Dataset dagi bazi sonli ustunlarni object formatidan integer va float formatiga o'tkazish
df['size'] = pd.to_numeric(df['size'], errors='coerce').astype('float64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    7565 non-null   object 
 1   district    7565 non-null   object 
 2   rooms       7565 non-null   int64  
 3   size        7564 non-null   float64
 4   level       7565 non-null   int64  
 5   max_levels  7565 non-null   int64  
 6   price       7466 non-null   Int64  
dtypes: Int64(1), float64(1), int64(3), object(2)
memory usage: 421.2+ KB


In [40]:
### Drop the 'location' column
house = df.drop('location', axis=1)

In [6]:
### Drop NaN values
house.dropna(inplace=True)

In [7]:
house.describe()

,rooms,size,level,max_levels,price
count,7465.000000,7465.000000,7465.000000,7465.000000,7465.0
mean,2.622103,113.537018,3.693771,6.023979,71323.672472
std,1.083155,1501.157996,2.236905,2.607103,640565.942591
min,1.000000,1.000000,1.000000,1.000000,2.0
25%,2.000000,50.000000,2.000000,4.000000,35000.0
50%,3.000000,65.000000,3.000000,5.000000,46500.0
75%,3.000000,85.000000,5.000000,9.000000,67000.0
max,10.000000,70000.000000,19.000000,25.000000,52000000.0


# Data preparation

In [41]:
train_set, test_set = train_test_split(house, test_size=0.2, random_state=42)

In [9]:
x_train = train_set.drop('price', axis=1)
y_train = train_set['price'].copy()

In [19]:
x_test = test_set.drop('price', axis=1)
y_test = test_set['price'].copy()

In [10]:
x_num = x_train.drop('district', axis=1)

# Create Pipeline

In [42]:
### pipeline for numeric columns
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [43]:
### Ful pipeline
from sklearn.compose import ColumnTransformer

num_attribs = list(x_num)
cat_attribs = ['district']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OrdinalEncoder(), cat_attribs)
])

In [44]:
x_prepared = full_pipeline.fit_transform(x_train)

In [45]:
x_test_prepared = full_pipeline.transform(x_test)

# Create a model

Linear regression

In [46]:
LR_model = LinearRegression()
LR_model.fit(x_prepared, y_train)

LinearRegression()

In [47]:
y_predict = LR_model.predict(x_test_prepared)
r2 = r2_score(y_predict, y_test)
mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
print("MAE=", mae)
print("RMSE=", np.sqrt(mse))
print("R2_score=", r2)

MAE= 67639.75885299627
RMSE= 1366181.8958744472
R2_score= -2389.849653110944


Random Forest

In [48]:
RF_model = RandomForestRegressor()
RF_model.fit(x_prepared, y_train)

RandomForestRegressor()

In [49]:
y_predict = RF_model.predict(x_test_prepared)
mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
r2 = r2_score(y_predict, y_test)
print("MAE=", mae)
print("RMSE=", np.sqrt(mse))
print("R2_score=", r2)

MAE= 62293.40418950059
RMSE= 1376331.562322515
R2_score= -67.15259407538967


Decision Tree

In [50]:
DT_model = DecisionTreeRegressor()
DT_model.fit(x_prepared, y_train)

DecisionTreeRegressor()

In [51]:
y_predict = DT_model.predict(x_test_prepared)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
r2 = r2_score(y_predict, y_test)
print("MAE=", mae)
print("RMSE=", np.sqrt(mse))
print("R2_score=", r2)

MAE= 69864.54630961202
RMSE= 1427013.536959952
R2_score= -10.934549287742026


GradientBoostingRegressor

In [52]:
GB_model = GradientBoostingRegressor()
GB_model.fit(x_prepared, y_train)

GradientBoostingRegressor()

In [53]:
y_predict = GB_model.predict(x_test_prepared)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
r2 = r2_score(y_predict, y_test)
print("MAE=", mae)
print("RMSE=", np.sqrt(mse))
print("R2_score=", r2)

MAE= 65849.95856494672
RMSE= 1392195.3661659895
R2_score= -26.434109098201375
